# Quick introduction

erddapy is a pure python package and can be installed with conda

```shell
conda install --channel conda-forge erddapy
```

or pip

```shell
pip install erddapy
```


First we need to instantiate the ERDDAP URL constructor for a server.
In these examples we will use
[https://standards.sensors.ioos.us/erddap/index.html](https://standards.sensors.ioos.us/erddap/index.html) and [https://erddap.ioos.us/erddap/index.html](https://erddap.ioos.us/erddap/index.html).

In [ ]:
from erddapy import ERDDAP

server = "https://standards.sensors.ioos.us/erddap"
e = ERDDAP(
    server=server,
    protocol="tabledap",
    response="csv",
)

Now we can populate the object a dataset id, variables of interest, and its
constraints (last week gliders). Use the method `to_pandas` to download the
csv(p) response, a comma separated values with units and explore the Dataframe.


In [ ]:
e.dataset_id = "org_cormp_cap2"

e.variables = [
    "time",
    "latitude",
    "longitude",
    "sea_water_temperature",
    "air_temperature",
]

e.constraints = {
    "time>=": "2000-01-01",
}


df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
).dropna()

df.head()

We can constraint the in time and space with relative constraints like in the
example below. For more ways to access the data please check the "Longer
introduction."


In [ ]:
server = "https://erddap.ioos.us/erddap"
e = ERDDAP(
    server=server,
    protocol="tabledap",
    response="csv",
)

e.dataset_id = "processed_asset_inventory"

# Get the box of the first 10 degrees bbox.
constraints = {
    "longitude<=": "min(longitude)+10",
    "longitude>=": "min(longitude)",
    "latitude<=": "min(latitude)+10",
    "latitude>=": "min(latitude)",
}


url = e.get_download_url(
    response="htmlTable",
    constraints=constraints,
)

print(url)

We can search all datasets with a set of constraints by setting `dataset_id` to `"allDatasets"`. Note that these variables are different than the ones available at the individual dataset level. For a reference of the possible variables to query all datasets see the `<server-url>/erddap/<protocol>/allDatasets.html` page, like [this one](https://erddap.ifremer.fr/erddap/tabledap/allDatasets.html) for the Ifremer ERDDAP server.

In [ ]:
e.dataset_id = "allDatasets"

e.variables = [
    "datasetID",
    "institution",
]


url = e.get_download_url(response="html")
print(url)

df = e.to_pandas(
    index_col="datasetID",
).dropna()


df.head()